In [1]:
"""
DESCRIPTION:
    This sample demonstrates how to use basic agent operations from
    the Azure Agents service using a synchronous client with Azure Monitor tracing.
    View the results in the "Tracing" tab in your Azure AI Foundry project page.

USAGE:
    python sample_agents_basics_with_azure_monitor_tracing.py

    Before running the sample:

    pip install azure-ai-projects azure-ai-agents azure-identity azure-monitor-opentelemetry

    Set these environment variables with your own values:
    1) PROJECT_ENDPOINT - The Azure AI Project endpoint, as found in the Overview
                          page of your Azure AI Foundry portal.
    2) MODEL_DEPLOYMENT_NAME - The deployment name of the AI model, as found under the "Name" column in
       the "Models + endpoints" tab in your Azure AI Foundry project.
    3) AZURE_TRACING_GEN_AI_CONTENT_RECORDING_ENABLED - Optional. Set to `true` to trace the content of chat
       messages, which may contain personal data. False by default.
    4) APPLICATIONINSIGHTS_CONNECTION_STRING - Set to the connection string of your Application Insights resource.
       This is used to send telemetry data to Azure Monitor. You can also get the connection string programmatically
       from AIProjectClient using the `telemetry.get_connection_string` method. A code sample showing how to do this
       can be found in the `sample_telemetry.py` file in the azure-ai-projects telemetry samples.
"""

import os, time
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import ListSortOrder
from azure.identity import DefaultAzureCredential
from azure.ai.agents import AgentsClient

import dotenv
# Load environment variables from .env file
dotenv.load_dotenv()

True

In [2]:
# [START enable_tracing]
from opentelemetry import trace
from azure.monitor.opentelemetry import configure_azure_monitor

# Enable Azure Monitor tracing
application_insights_connection_string = os.environ["APPLICATIONINSIGHTS_CONNECTION_STRING"]
configure_azure_monitor(connection_string=application_insights_connection_string)

scenario = os.path.basename("AI-AGENT_TRACING_MONITORING-2.ipynb")
tracer = trace.get_tracer("monitoring-sample", "1.0.0")

In [3]:
project_client = AgentsClient(
    endpoint=os.getenv("PROJECT_ENDPOINT"),
    credential=DefaultAzureCredential(),
)

In [4]:
with tracer.start_as_current_span(scenario):
    with project_client:

        # [END enable_tracing]
        agent = project_client.create_agent(
            model=os.environ["MODEL_DEPLOYMENT_NAME"], name="my-agent-with-tracing", instructions="You are helpful agent"
        )
        print(f"Created agent, agent ID: {agent.id}")

        thread = project_client.threads.create()
        print(f"Created thread, thread ID: {thread.id}")

        message = project_client.messages.create(
            thread_id=thread.id, role="user", content="Hello, what is the easiest way to learn Python?"
        )
        print(f"Created message, message ID: {message.id}")

        run = project_client.runs.create_and_process(thread_id=thread.id, agent_id=agent.id)
        print(f"Run completed with status: {run.status}")

        project_client.delete_agent(agent.id)
        print("Deleted agent")

        messages = project_client.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)
        for msg in messages:
            if msg.text_messages:
                last_text = msg.text_messages[-1]
                print(f"{msg.role}: {last_text.text.value}")

Created agent, agent ID: asst_HdlLfKxLENErvUTB7ByszZUH
Created thread, thread ID: thread_gN8F8uYBR4t6p68NgAQg3g68
Created message, message ID: msg_m20qbDToDywplnA08QG90qve
Run completed with status: RunStatus.COMPLETED
Deleted agent
MessageRole.USER: Hello, what is the easiest way to learn Python?
MessageRole.AGENT: Hello! Learning Python can be enjoyable and straightforward if you approach it step by step. Here’s an **easy way to get started with Python**:

### 1. **Set Up Your Environment**
- Download and install [Python](https://www.python.org/downloads/) (if it’s not already on your computer).
- Use a beginner-friendly code editor, such as [Thonny](https://thonny.org/) or [VS Code](https://code.visualstudio.com/).

### 2. **Follow an Interactive Tutorial**
- Try websites like [Codecademy](https://www.codecademy.com/learn/learn-python-3), [SoloLearn](https://www.sololearn.com/Course/Python/), or [W3Schools Python Tutorial](https://www.w3schools.com/python/). They let you write and t